# Final Notebook: Combine, Clean, and Export Election Data

In [1]:
import pandas as pd

files = [
    "pdy_de.xlsx", "pdy_fi.xlsx", "pdy_ie.xlsx", "pdy_is.xlsx",
    "pdy_it.xlsx", "pdy_lu.xlsx", "pdy_nl.xlsx", "pdy_no.xlsx",
    "pdy_pt.xlsx", "pdy_se.xlsx", "pdy_au.xlsx", "pdy_be.xlsx",
    "pdy_at.xlsx", "pdy_dk.xlsx"
]

def process_countryID_file(file_path, countryID_code):
    try:
        df = pd.read_excel(file_path, sheet_name="parlvotes_lh", header=None)
    except:
        return None

    start_col = 2
    block_width = 8
    var_row = 9
    data_start_row = 10

    id_df = df.iloc[data_start_row:, [0, 1]].copy()
    id_df.columns = ["party_id", "party_name"]

    date_col_indices = []
    for col in range(start_col, df.shape[1]):
        for row in [0, 1]:
            val = df.iloc[row, col]
            if isinstance(val, str):
                val = val.strip()
            if pd.notnull(val) and val != "":
                try:
                    pd.to_datetime(val)
                    date_col_indices.append(col)
                    break
                except:
                    continue

    unique_start_cols = sorted(set(date_col_indices))
    blocks = []

    for col in unique_start_cols:
        block = df.iloc[data_start_row:, col:col+block_width].copy()
        block.columns = df.iloc[var_row, col:col+block_width].values

        date_val = None
        for row in [0, 1]:
            val = df.iloc[row, col]
            if isinstance(val, str):
                val = val.strip()
            if pd.notnull(val) and val != "":
                try:
                    date_val = pd.to_datetime(val)
                    break
                except:
                    continue

        if date_val is None:
            continue

        block["election_date"] = date_val
        block["partyID"] = id_df["party_id"].values
        block["party_name"] = id_df["party_name"].values
        blocks.append(block)

    if not blocks:
        return None

    df_all = pd.concat(blocks, axis=0).reset_index(drop=True)
    df_all = df_all.dropna(axis=1, how='all')
    df_all = df_all.dropna(subset=["election_date"])
    df_all = df_all.drop(columns=["Name of party (if different from column b)"], errors='ignore')

    seat_cols = [col for col in df_all.columns if isinstance(col, str) and 'seats' in col.lower()]
    numeric_seat_cols = df_all[seat_cols].select_dtypes(include='number') if seat_cols else pd.DataFrame()

    if not numeric_seat_cols.empty:
        df_all = df_all[numeric_seat_cols.sum(axis=1) > 0]

    df_all = df_all.rename(columns={
        "# of Votes_type1": "votes",
        "% of Votes_type1": "share_votes",
        "Change in % of Votes_type1": "change_share_votes",
        "Seats_type1": "seats",
        "% of Seats_type1": "share_seats",
        "Change in % of Seats_type1": "change_share_seats",
        "party_id": "partyID"
    })

    df_all["year"] = pd.to_datetime(df_all["election_date"]).dt.year
    df_all["countryID"] = countryID_code.upper()

    return df_all

# Combine all countries
all_dfs = []
for file_name in files:
    countryID_code = file_name.split("_")[1].split(".")[0]
    file_path = file_name
    result = process_countryID_file(file_path, countryID_code)
    if result is not None:
        all_dfs.append(result)

combined_df = pd.concat(all_dfs, axis=0).reset_index(drop=True)

# Remove rows where seats is NaN or 0
if "seats" in combined_df.columns:
    combined_df = combined_df[combined_df["seats"].notna() & (combined_df["seats"] != 0)]

# Drop empty seat columns
seat_cols = [col for col in combined_df.columns if isinstance(col, str) and 'seats' in col.lower()]
empty_seat_cols = [col for col in seat_cols if combined_df[col].isna().all()]
combined_df = combined_df.drop(columns=empty_seat_cols)

# Drop ballot column if present
combined_df = combined_df.drop(columns=["Ballot #--may not be in most countries"], errors="ignore")

# Sort by countryID and year
combined_df = combined_df.sort_values(by=["countryID", "year"]).reset_index(drop=True)

# Save final file
combined_df.to_excel("election_data.xlsx", index=False)
combined_df.head()

d:\Users\lucas\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
d:\Users\lucas\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
d:\Users\lucas\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
d:\Users\lucas\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
d:\Users\lucas\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
d:\Users\lucas\anaconda3\Lib\site-packages\openpyx

,votes,share_votes,change_share_votes,seats,share_seats,change_share_seats,election_date,partyID,party_name,year,countryID
0,1617804,0.349,-0.079,65,0.355,-0.082,1994-10-09,at_spo01,Social Democratic Party of Austria (Sozialdem...,1994,AT
1,1281846,0.277,-0.044,52,0.284,-0.044,1994-10-09,at_ovp01,Austrian People's Party (Österreichische Volk...,1994,AT
2,1042332,0.225,0.059,42,0.23,0.049,1994-10-09,at_fpo01,Freedom Party of Austria (Freiheitliche Partei...,1994,AT
3,338538,0.073,0.025,13,0.071,0.016,1994-10-09,at_ga01,The Greens-Green Alternative (Die Grünen-Die G...,1994,AT
4,276580,0.06,0.06,11,0.06,0.033,1994-10-09,at_lif01,"Liberal Forum (Liberales Forum, LIF)",1994,AT
